In [1]:
import pandas as pd
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('train.csv')
df = df.rename(columns = {'tweet_text':'text', 'check_worthiness':'labels'})
train, test = train_test_split(df, test_size=0.1, shuffle=False)
test = test.drop(['index', 'topic_id', 'tweet_id', 'tweet_url', 'claim'], 1)

In [3]:
model = ClassificationModel('roberta', 'roberta-base', args={"num_train_epochs": 40, 
                                                                  "fp16_opt_level" : 'O0',
                                                                 "max_seq_length" : 300,
                                                                  "overwrite_output_dir": True, 
                                                                  "save_eval_checkpoints": False, 
                                                                  "save_model_every_epoch": False})

In [4]:
model.train_model(train)


Selected optimization level O0:  Pure FP32 training.

Defaults for this optimization level are:
enabled                : True
opt_level              : O0
cast_model_type        : torch.float32
patch_torch_functions  : False
keep_batchnorm_fp32    : None
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O0
cast_model_type        : torch.float32
patch_torch_functions  : False
keep_batchnorm_fp32    : None
master_weights         : False
loss_scale             : 1.0


Running loss: 0.125237


Running loss: 0.563463


Running loss: 0.163172


Running loss: 0.017699


...

Running loss: 0.000021


Running loss: 0.000020


Running loss: 0.000016


Running loss: 0.000021


Running loss: 0.000019



In [15]:
model.eval_model(test)

({'mcc': 0.47570670890561023,
  'tp': 28,
  'tn': 33,
  'fp': 8,
  'fn': 14,
  'eval_loss': 2.531253793925895},
 array([[-6.013085  ,  5.213481  ],
        [ 6.0558877 , -5.4494047 ],
        [-5.811372  ,  5.0082073 ],
        [ 5.982779  , -5.2680864 ],
        [-6.0178537 ,  5.2243805 ],
        [ 6.0769916 , -5.43201   ],
        [ 6.072044  , -5.4381638 ],
        [-5.5146837 ,  4.766255  ],
        [-6.02756   ,  5.23622   ],
        [ 6.0659876 , -5.4385996 ],
        [-6.01667   ,  5.227081  ],
        [ 5.6447535 , -4.8945794 ],
        [ 6.0697412 , -5.390399  ],
        [ 5.8644323 , -5.111882  ],
        [ 6.0575256 , -5.442384  ],
        [-5.8522654 ,  5.078145  ],
        [ 6.0450544 , -5.4166484 ],
        [ 2.7410483 , -2.1364796 ],
        [ 6.093438  , -5.4281273 ],
        [ 0.84632796, -0.47330225],
        [-4.4058213 ,  3.7791176 ],
        [ 4.2651234 , -3.4245312 ],
        [-5.9435244 ,  5.142928  ],
        [ 5.2725782 , -4.4160485 ],
        [ 6.0640736 , -5

In [12]:
temp = pd.read_csv('test.csv')
temp

,index,topic_id,tweet_id,tweet_url,tweet_text
0,1,covid-19,1237160250513522688,https://twitter.com/user/status/12371602505135...,potus wanted everyone know close contact gaetz...
1,2,covid-19,1237125962871037953,https://twitter.com/user/status/12371259628710...,would prefer lead nation response growing coro...
2,3,covid-19,1237207721604235264,https://twitter.com/user/status/12372077216042...,really really really really really really real...
3,4,covid-19,1237178597024108552,https://twitter.com/user/status/12371785970241...,baldfaced lie replouiegohmert selfquarantine c...
4,5,covid-19,1237049051058561024,https://twitter.com/user/status/12370490510585...,live daily medium briefing covid drtedros coro...
...,...,...,...,...,...
135,136,covid-19,1237094362544209920,https://twitter.com/user/status/12370943625442...,corona virus spreading fast spread dont natura...
136,137,covid-19,1237063585580318720,https://twitter.com/user/status/12370635855803...,market crashing tourism dying travel reduced a...
137,138,covid-19,1237159267070664705,https://twitter.com/user/status/12371592670706...,pray palestine state emergency case corona vi...
138,139,covid-19,1236972990492897281,https://twitter.com/user/status/12369729904928...,advisory student class suspension made safety ...


In [13]:
model = ClassificationModel('roberta', 'outputs/')

In [14]:
results = model.predict(temp['tweet_text'])

In [15]:
from scipy.special import softmax

probabilities = softmax(results[1], axis=1)

scores = []

preds_df = pd.DataFrame(probabilities[:,1])
preds_df.columns = ['score']

submit = pd.merge(temp, preds_df, left_index=True, right_index=True)
submit['run_id'] = "T1-eng-SSN-Transformer"
    
#temp['values'] = results[0]
#temp['predictions'] = scores
#temp = temp[temp['values'] == 1]
#temp = temp.drop(['index', 'tweet_url', 'tweet_text'], 1)
#temp['run_id'] = 'T2-eng-SSN-Roberta'
#temp = temp.sort_values(by =['predictions'], ascending = 0) 

In [16]:
submit

,index,topic_id,tweet_id,tweet_url,tweet_text,score,run_id
0,1,covid-19,1237160250513522688,https://twitter.com/user/status/12371602505135...,potus wanted everyone know close contact gaetz...,0.000010,T1-eng-SSN-Transformer
1,2,covid-19,1237125962871037953,https://twitter.com/user/status/12371259628710...,would prefer lead nation response growing coro...,0.000010,T1-eng-SSN-Transformer
2,3,covid-19,1237207721604235264,https://twitter.com/user/status/12372077216042...,really really really really really really real...,0.000020,T1-eng-SSN-Transformer
3,4,covid-19,1237178597024108552,https://twitter.com/user/status/12371785970241...,baldfaced lie replouiegohmert selfquarantine c...,0.999979,T1-eng-SSN-Transformer
4,5,covid-19,1237049051058561024,https://twitter.com/user/status/12370490510585...,live daily medium briefing covid drtedros coro...,0.000010,T1-eng-SSN-Transformer
...,...,...,...,...,...,...,...
135,136,covid-19,1237094362544209920,https://twitter.com/user/status/12370943625442...,corona virus spreading fast spread dont natura...,0.999981,T1-eng-SSN-Transformer
136,137,covid-19,1237063585580318720,https://twitter.com/user/status/12370635855803...,market crashing tourism dying travel reduced a...,0.999979,T1-eng-SSN-Transformer
137,138,covid-19,1237159267070664705,https://twitter.com/user/status/12371592670706...,pray palestine state emergency case corona vi...,0.999987,T1-eng-SSN-Transformer
138,139,covid-19,1236972990492897281,https://twitter.com/user/status/12369729904928...,advisory student class suspension made safety ...,0.000010,T1-eng-SSN-Transformer


In [17]:
submit = submit.drop(['index', 'tweet_url', 'tweet_text'], 1)
submit

,topic_id,tweet_id,score,run_id
0,covid-19,1237160250513522688,0.000010,T1-eng-SSN-Transformer
1,covid-19,1237125962871037953,0.000010,T1-eng-SSN-Transformer
2,covid-19,1237207721604235264,0.000020,T1-eng-SSN-Transformer
3,covid-19,1237178597024108552,0.999979,T1-eng-SSN-Transformer
4,covid-19,1237049051058561024,0.000010,T1-eng-SSN-Transformer
...,...,...,...,...
135,covid-19,1237094362544209920,0.999981,T1-eng-SSN-Transformer
136,covid-19,1237063585580318720,0.999979,T1-eng-SSN-Transformer
137,covid-19,1237159267070664705,0.999987,T1-eng-SSN-Transformer
138,covid-19,1236972990492897281,0.000010,T1-eng-SSN-Transformer


In [18]:
submit.to_csv('t1-roberta.tsv', sep='\t', index=False)